In [1]:
# =============================================================================
# COMBINE THECB ENROLLMENT + SAT/ACT READINESS DATA
# =============================================================================

import pandas as pd
import numpy as np

print("="*70)
print("COMBINING THECB ENROLLMENT + SAT/ACT READINESS DATA")
print("="*70)

# =============================================================================
# STEP 1: Load Both Datasets
# =============================================================================

print("\nLoading THECB enrollment data...")
thecb = pd.read_csv('county_dominance_data.csv')
print(f"✅ Loaded {len(thecb)} counties from THECB data")

print("\nLoading SAT/ACT readiness data...")
sat_act = pd.read_csv('county_sat_act_rates_only.csv')
print(f"✅ Loaded {len(sat_act)} counties from SAT/ACT data")

# =============================================================================
# STEP 2: Merge on County
# =============================================================================

print("\n" + "="*70)
print("MERGING DATASETS")
print("="*70)

# Merge
combined = thecb.merge(
    sat_act[['County', 'College_Ready_Rate', 'TSI_Ready_Rate', 'Participation_Rate']],
    on='County',
    how='left'
)

print(f"\n✅ Combined dataset: {len(combined)} counties")
print(f"   Counties with both datasets: {combined['College_Ready_Rate'].notna().sum()}")
print(f"   Counties with only THECB data: {combined['College_Ready_Rate'].isna().sum()}")

# =============================================================================
# STEP 3: Calculate Strategic Metrics
# =============================================================================

print("\n" + "="*70)
print("CALCULATING STRATEGIC RECRUITMENT METRICS")
print("="*70)

# 1. High Academic Performance Flag
combined['High_Academic_Performance'] = combined['College_Ready_Rate'] >= 25  # Top quartile

# 2. Low TXST Presence Flag
combined['Low_TXST_Presence'] = combined['TXST_Market_Share'] < 10

# 3. Large Market Flag
combined['Large_Market'] = combined['Total_All_Schools'] > 2000

# 4. High Test Participation Flag
combined['High_Participation'] = combined['Participation_Rate'] >= 70

# 5. PRIME TARGET: High performance + Low TXST + Large market
combined['Prime_Target'] = (
    combined['High_Academic_Performance'] & 
    combined['Low_TXST_Presence'] & 
    combined['Large_Market']
)

# 6. Calculate Opportunity Score (weighted composite)
# Factors: Academic performance (40%), Market size (30%), Low TXST share (20%), Participation (10%)

# Normalize each factor to 0-100 scale
combined['Academic_Score'] = (combined['College_Ready_Rate'] / 50 * 100).clip(0, 100).fillna(0)
combined['Market_Score'] = (np.log1p(combined['Total_All_Schools']) / np.log1p(combined['Total_All_Schools'].max()) * 100).fillna(0)
combined['TXST_Gap_Score'] = (100 - combined['TXST_Market_Share']).clip(0, 100).fillna(0)
combined['Participation_Score'] = combined['Participation_Rate'].fillna(0)

# Composite opportunity score
combined['Strategic_Opportunity_Score'] = (
    combined['Academic_Score'] * 0.40 +
    combined['Market_Score'] * 0.30 +
    combined['TXST_Gap_Score'] * 0.20 +
    combined['Participation_Score'] * 0.10
)

# 7. Classify opportunity tiers
def classify_opportunity(row):
    if pd.isna(row['College_Ready_Rate']):
        return 'No Academic Data'
    elif row['Prime_Target']:
        return 'Tier 1: Prime Target'
    elif row['High_Academic_Performance'] and row['Low_TXST_Presence']:
        return 'Tier 2: Strong Opportunity'
    elif row['High_Academic_Performance']:
        return 'Tier 3: High Performers (TXST Present)'
    elif row['Large_Market'] and row['Low_TXST_Presence']:
        return 'Tier 4: Volume Play'
    else:
        return 'Tier 5: Maintain/Monitor'

combined['Opportunity_Tier'] = combined.apply(classify_opportunity, axis=1)

print("\n✅ Strategic metrics calculated!")

# =============================================================================
# STEP 4: Summary Statistics
# =============================================================================

print("\n" + "="*70)
print("OPPORTUNITY TIER DISTRIBUTION")
print("="*70)

print(combined['Opportunity_Tier'].value_counts().sort_index())

# =============================================================================
# STEP 5: Top Targets Analysis
# =============================================================================

print("\n" + "="*70)
print("🎯 TIER 1: PRIME RECRUITMENT TARGETS")
print("="*70)
print("(High Academic Performance + Large Market + Low TXST Presence)")
print()

tier1 = combined[combined['Opportunity_Tier'] == 'Tier 1: Prime Target'].sort_values(
    'Strategic_Opportunity_Score', ascending=False
)

if len(tier1) > 0:
    print(f"Found {len(tier1)} prime target counties:\n")
    print(f"{'County':<15} {'College Ready':<13} {'Total Market':<12} {'TXST Share':<11} {'Opp Score':<10}")
    print("-" * 70)
    
    for idx, row in tier1.iterrows():
        print(f"{row['County']:<15} {row['College_Ready_Rate']:>10.1f}%  {row['Total_All_Schools']:>10,.0f}  "
              f"{row['TXST_Market_Share']:>8.1f}%  {row['Strategic_Opportunity_Score']:>8.0f}/100")
    
    print(f"\n💰 POTENTIAL IMPACT:")
    print(f"   Total market: {tier1['Total_All_Schools'].sum():,.0f} students")
    print(f"   Current TXST: {tier1['TXST'].sum():,.0f} students ({tier1['TXST'].sum()/tier1['Total_All_Schools'].sum()*100:.1f}%)")
    print(f"   If TXST reached 15% share: {tier1['Total_All_Schools'].sum() * 0.15:,.0f} students")
    print(f"   Growth potential: +{(tier1['Total_All_Schools'].sum() * 0.15) - tier1['TXST'].sum():,.0f} students")
else:
    print("No Tier 1 targets found with current thresholds")

print("\n" + "="*70)
print("🎯 TIER 2: STRONG OPPORTUNITIES")
print("="*70)
print("(High Academic Performance + Low TXST Presence)")
print()

tier2 = combined[combined['Opportunity_Tier'] == 'Tier 2: Strong Opportunity'].sort_values(
    'Strategic_Opportunity_Score', ascending=False
).head(15)

if len(tier2) > 0:
    print(f"Top 15 Tier 2 counties:\n")
    print(f"{'County':<15} {'College Ready':<13} {'Total Market':<12} {'TXST Share':<11} {'Opp Score':<10}")
    print("-" * 70)
    
    for idx, row in tier2.iterrows():
        print(f"{row['County']:<15} {row['College_Ready_Rate']:>10.1f}%  {row['Total_All_Schools']:>10,.0f}  "
              f"{row['TXST_Market_Share']:>8.1f}%  {row['Strategic_Opportunity_Score']:>8.0f}/100")

# =============================================================================
# STEP 6: Regional Analysis
# =============================================================================

print("\n" + "="*70)
print("📊 PERFORMANCE BY DOMINANT SCHOOL")
print("="*70)

perf_by_school = combined[combined['College_Ready_Rate'].notna()].groupby('Dominant_School').agg({
    'College_Ready_Rate': 'mean',
    'TSI_Ready_Rate': 'mean',
    'Participation_Rate': 'mean',
    'Total_All_Schools': 'sum',
    'County': 'count'
}).round(1)

perf_by_school.columns = ['Avg College Ready %', 'Avg TSI %', 'Avg Participation %', 'Total Students', 'Counties']
perf_by_school = perf_by_school.sort_values('Avg College Ready %', ascending=False)

print(perf_by_school)

# =============================================================================
# STEP 7: Save Combined Dataset
# =============================================================================

print("\n" + "="*70)
print("SAVING COMBINED DATASET")
print("="*70)

combined.to_csv('county_combined_enrollment_readiness.csv', index=False)
print("✅ Saved: county_combined_enrollment_readiness.csv")

# Also save just the prime targets
tier1.to_csv('prime_recruitment_targets.csv', index=False)
print("✅ Saved: prime_recruitment_targets.csv")

# Save all tiers for reference
all_tiers = combined[combined['Opportunity_Tier'] != 'No Academic Data'].sort_values(
    'Strategic_Opportunity_Score', ascending=False
)
all_tiers.to_csv('recruitment_targets_all_tiers.csv', index=False)
print("✅ Saved: recruitment_targets_all_tiers.csv")

# =============================================================================
# STEP 8: Key Insights
# =============================================================================

print("\n" + "="*70)
print("🔑 KEY INSIGHTS")
print("="*70)

print(f"\n1. DATA COVERAGE:")
print(f"   • {combined['College_Ready_Rate'].notna().sum()} counties have both enrollment + readiness data")
print(f"   • {combined['College_Ready_Rate'].isna().sum()} counties lack readiness data")

print(f"\n2. PRIME TARGETS:")
print(f"   • {len(tier1)} Tier 1 counties (top priority)")
print(f"   • {len(combined[combined['Opportunity_Tier'] == 'Tier 2: Strong Opportunity'])} Tier 2 counties (strong opportunities)")
print(f"   • Combined opportunity: {(tier1['Total_All_Schools'].sum() + combined[combined['Opportunity_Tier'] == 'Tier 2: Strong Opportunity']['Total_All_Schools'].sum()):,.0f} students")

print(f"\n3. ACADEMIC PERFORMANCE:")
avg_college_ready = combined['College_Ready_Rate'].mean()
print(f"   • Statewide avg college readiness: {avg_college_ready:.1f}%")
print(f"   • TXST stronghold counties avg: {combined[combined['Dominant_School'] == 'TXST']['College_Ready_Rate'].mean():.1f}%")
if len(tier1) > 0:
    print(f"   • Prime target counties avg: {tier1['College_Ready_Rate'].mean():.1f}%")

print(f"\n4. TEST PARTICIPATION:")
print(f"   • Statewide avg participation: {combined['Participation_Rate'].mean():.1f}%")
print(f"   • High participation counties: {(combined['Participation_Rate'] >= 80).sum()}")

print("\n" + "="*70)
print("✅ COMBINED ANALYSIS COMPLETE!")
print("="*70)

COMBINING THECB ENROLLMENT + SAT/ACT READINESS DATA

Loading THECB enrollment data...
✅ Loaded 265 counties from THECB data

Loading SAT/ACT readiness data...
✅ Loaded 252 counties from SAT/ACT data

MERGING DATASETS

✅ Combined dataset: 265 counties
   Counties with both datasets: 229
   Counties with only THECB data: 36

CALCULATING STRATEGIC RECRUITMENT METRICS

✅ Strategic metrics calculated!

OPPORTUNITY TIER DISTRIBUTION
Opportunity_Tier
No Academic Data                           36
Tier 1: Prime Target                        6
Tier 2: Strong Opportunity                 20
Tier 3: High Performers (TXST Present)      2
Tier 4: Volume Play                        40
Tier 5: Maintain/Monitor                  161
Name: count, dtype: int64

🎯 TIER 1: PRIME RECRUITMENT TARGETS
(High Academic Performance + Large Market + Low TXST Presence)

Found 6 prime target counties:

County          College Ready Total Market TXST Share  Opp Score 
---------------------------------------------------

In [2]:
# =============================================================================
# STUNNING STRATEGIC OPPORTUNITY TIER MAP (POSTER-READY)
# =============================================================================

import pandas as pd
import geopandas as gpd
import plotly.graph_objects as go
import json

print("="*70)
print("CREATING BEAUTIFUL STRATEGIC OPPORTUNITY MAP")
print("="*70)

# Load data
combined = pd.read_csv('county_combined_enrollment_readiness.csv')
counties = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_us_county_20m.zip')
texas_counties = counties[counties['STATEFP'] == '48'].copy()
texas_counties['County'] = texas_counties['NAME'].str.upper()
texas_merged = texas_counties.merge(combined, on='County', how='left')

geojson = json.loads(texas_merged.to_json())

# Tier mapping
tier_mapping = {
    'Tier 1: Prime Target': 6,
    'Tier 2: Strong Opportunity': 5,
    'Tier 3: High Performers (TXST Present)': 4,
    'Tier 4: Volume Play': 3,
    'Tier 5: Maintain/Monitor': 2,
    'No Academic Data': 1
}
texas_merged['Tier_Numeric'] = texas_merged['Opportunity_Tier'].map(tier_mapping)

# GORGEOUS color palette
tier_colors = {
    6: '#FFB700',    # Bright gold - Tier 1
    5: '#FF6B6B',    # Coral red - Tier 2
    4: '#4ECDC4',    # Turquoise - Tier 3
    3: '#95E1D3',    # Mint - Tier 4
    2: '#E8E8E8',    # Light gray - Tier 5
    1: '#BDBDBD'     # Medium gray - No data
}

colorscale = []
for i in range(1, 7):
    colorscale.append([(i-1)/6, tier_colors[i]])
    colorscale.append([i/6, tier_colors[i]])

# =============================================================================
# CREATE MAIN MAP
# =============================================================================

fig = go.Figure()

# Main choropleth with elegant styling
fig.add_trace(go.Choropleth(
    geojson=geojson,
    locations=texas_merged.index,
    z=texas_merged['Tier_Numeric'],
    colorscale=colorscale,
    zmin=1,
    zmax=6,
    showscale=False,
    marker_line_color='#2C3E50',
    marker_line_width=0.8,
    hovertemplate=
        '<b style="font-size:20px; color:#2C3E50">%{customdata[0]} County</b><br><br>' +
        '<span style="font-size:16px; color:#E74C3C">●</span> <b style="font-size:15px">%{customdata[1]}</b><br><br>' +
        '<b style="color:#34495E">📊 Academic Performance</b><br>' +
        '  College Ready: <b>%{customdata[2]:.1f}%</b><br>' +
        '  TSI Ready: <b>%{customdata[3]:.1f}%</b><br>' +
        '  Participation: <b>%{customdata[4]:.1f}%</b><br><br>' +
        '<b style="color:#34495E">🎓 Market Data</b><br>' +
        '  Total Students: <b>%{customdata[5]:,.0f}</b><br>' +
        '  TXST Share: <b>%{customdata[7]:.1f}%</b> (%{customdata[6]:,.0f} students)<br>' +
        '  Current Leader: <b>%{customdata[8]}</b><br><br>' +
        '<b style="color:#E74C3C">🎯 Opportunity Score: %{customdata[9]:.0f}/100</b>' +
        '<extra></extra>',
    customdata=texas_merged[[
        'County', 'Opportunity_Tier', 'College_Ready_Rate', 'TSI_Ready_Rate',
        'Participation_Rate', 'Total_All_Schools', 'TXST', 'TXST_Market_Share',
        'Dominant_School', 'Strategic_Opportunity_Score'
    ]].fillna(0).values
))

# TXST campus - elegant star
fig.add_trace(go.Scattergeo(
    lon=[-97.9],
    lat=[29.9],
    mode='markers',
    marker=dict(
        size=45,
        color='#501214',
        symbol='star',
        line=dict(width=4, color='#FFB700'),
        opacity=1
    ),
    hovertemplate='<b style="font-size:18px; color:#501214">Texas State University</b><br><span style="color:#666">San Marcos, Texas</span><extra></extra>',
    showlegend=False
))

# Add elegant label for TXST
fig.add_trace(go.Scattergeo(
    lon=[-97.9],
    lat=[29.3],
    mode='text',
    text=['<b>TEXAS STATE<br>UNIVERSITY</b>'],
    textfont=dict(size=13, color='#501214', family='Inter, sans-serif'),
    hoverinfo='skip',
    showlegend=False
))

# Tier 1 counties - stylish markers with glow effect
tier1_counties = texas_merged[texas_merged['Opportunity_Tier'] == 'Tier 1: Prime Target']

for idx, row in tier1_counties.iterrows():
    centroid = row['geometry'].centroid
    
    # Outer glow
    fig.add_trace(go.Scattergeo(
        lon=[centroid.x],
        lat=[centroid.y],
        mode='markers',
        marker=dict(size=35, color='#FFB700', opacity=0.3),
        hoverinfo='skip',
        showlegend=False
    ))
    
    # Inner circle
    fig.add_trace(go.Scattergeo(
        lon=[centroid.x],
        lat=[centroid.y],
        mode='markers',
        marker=dict(
            size=18,
            color='#E74C3C',
            symbol='circle',
            line=dict(width=3, color='white')
        ),
        hoverinfo='skip',
        showlegend=False
    ))
    
    # Elegant label
    fig.add_trace(go.Scattergeo(
        lon=[centroid.x],
        lat=[centroid.y + 0.3],
        mode='text',
        text=[f'<b>{row["County"]}</b>'],
        textfont=dict(size=11, color='#2C3E50', family='Inter, sans-serif'),
        hoverinfo='skip',
        showlegend=False
    ))

fig.update_geos(
    fitbounds="locations",
    visible=False,
    bgcolor='#FAFAFA'
)

fig.update_layout(
    title=dict(
        text='<b style="font-size:32px; color:#2C3E50; letter-spacing:1px">Strategic Recruitment Opportunities</b><br>' +
             '<span style="font-size:16px; color:#7F8C8D; font-weight:300">Identifying High-Performing Markets for Texas State University Growth</span>',
        x=0.5,
        xanchor='center',
        y=0.98,
        yanchor='top',
        font=dict(family='Inter, -apple-system, sans-serif')
    ),
    height=1200,
    width=2000,
    paper_bgcolor='#FAFAFA',
    geo=dict(
        scope='usa',
        center=dict(lon=-99, lat=31),
        projection_scale=3.5,
        bgcolor='#FAFAFA'
    ),
    margin=dict(l=20, r=250, t=140, b=20)
)

# BEAUTIFUL custom legend
legend_data = [
    {
        'tier': 'Tier 1: Prime Target',
        'color': '#FFB700',
        'count': 6,
        'desc': 'High performance • Large market • Low TXST presence',
        'icon': '⭐'
    },
    {
        'tier': 'Tier 2: Strong Opportunity',
        'color': '#FF6B6B',
        'count': 20,
        'desc': 'High academic performance • Low TXST presence',
        'icon': '🎯'
    },
    {
        'tier': 'Tier 3: High Performers',
        'color': '#4ECDC4',
        'count': 2,
        'desc': 'Strong academics • TXST already present',
        'icon': '✓'
    },
    {
        'tier': 'Tier 4: Volume Play',
        'color': '#95E1D3',
        'count': 40,
        'desc': 'Large markets • Growth opportunity',
        'icon': '○'
    },
    {
        'tier': 'Tier 5: Maintain',
        'color': '#E8E8E8',
        'count': 161,
        'desc': 'Monitor for changes',
        'icon': '·'
    }
]

# Add annotation box for legend
legend_text = '<b style="font-size:16px; color:#2C3E50">OPPORTUNITY TIERS</b><br><br>'

for item in legend_data:
    legend_text += (
        f'<span style="color:{item["color"]}; font-size:20px">●</span> '
        f'<b style="color:#2C3E50">{item["tier"]}</b> '
        f'<span style="color:#95A5A6">({item["count"]} counties)</span><br>'
        f'<span style="color:#7F8C8D; font-size:12px; margin-left:20px">{item["desc"]}</span><br><br>'
    )

fig.add_annotation(
    x=1.15,
    y=0.75,
    xref='paper',
    yref='paper',
    text=legend_text,
    showarrow=False,
    align='left',
    bgcolor='white',
    bordercolor='#2C3E50',
    borderwidth=2,
    borderpad=15,
    font=dict(size=12, family='Inter, sans-serif')
)

# Add insight box
insight_text = (
    '<b style="font-size:14px; color:#E74C3C">KEY INSIGHT</b><br><br>'
    '<span style="color:#2C3E50">The 6 Tier 1 counties represent</span><br>'
    '<b style="font-size:20px; color:#E74C3C">+11,017 students</b><br>'
    '<span style="color:#7F8C8D; font-size:11px">growth potential if TXST reaches 15% market share</span><br><br>'
    '<b style="font-size:18px; color:#FFB700">85,560</b> <span style="color:#7F8C8D">total students</span><br>'
    '<b style="font-size:18px; color:#4ECDC4">29.9%</b> <span style="color:#7F8C8D">avg. college ready</span><br>'
    '<b style="font-size:18px; color:#95E1D3">2.1%</b> <span style="color:#7F8C8D">current TXST share</span>'
)

fig.add_annotation(
    x=1.15,
    y=0.35,
    xref='paper',
    yref='paper',
    text=insight_text,
    showarrow=False,
    align='left',
    bgcolor='#FFF9E6',
    bordercolor='#FFB700',
    borderwidth=3,
    borderpad=15,
    font=dict(size=12, family='Inter, sans-serif')
)

fig.write_html('texas_opportunity_BEAUTIFUL.html')
fig.write_image('texas_opportunity_BEAUTIFUL.png', width=3000, height=1800, scale=2)

print("\n✅ Beautiful main map saved!")

# =============================================================================
# CREATE STUNNING TIER 1 SPOTLIGHT MAP
# =============================================================================

print("\nCreating Tier 1 spotlight map...")

fig2 = go.Figure()

# Base map - very subtle
fig2.add_trace(go.Choropleth(
    geojson=geojson,
    locations=texas_merged.index,
    z=[1]*len(texas_merged),
    colorscale=[[0, '#F8F9FA'], [1, '#F8F9FA']],
    showscale=False,
    marker_line_color='#DEE2E6',
    marker_line_width=0.5,
    hoverinfo='skip'
))

# Tier 1 highlight - gorgeous gold
tier1_numeric = texas_merged['Opportunity_Tier'].map({'Tier 1: Prime Target': 1}).fillna(0)

fig2.add_trace(go.Choropleth(
    geojson=geojson,
    locations=texas_merged.index,
    z=tier1_numeric,
    colorscale=[[0, 'rgba(0,0,0,0)'], [0.5, '#FFE082'], [1, '#FFB700']],
    showscale=False,
    marker_line_color='#E65100',
    marker_line_width=4,
    hovertemplate=
        '<b style="font-size:22px; color:#E65100">%{customdata[0]} County</b><br><br>' +
        '<span style="font-size:18px; color:#FFB700">⭐ TIER 1 PRIME TARGET</span><br><br>' +
        '<b style="color:#2C3E50">Academic Performance</b><br>' +
        '  College Ready: <b style="color:#27AE60">%{customdata[1]:.1f}%</b><br><br>' +
        '<b style="color:#2C3E50">Market Size</b><br>' +
        '  Total Students: <b style="color:#3498DB">%{customdata[2]:,.0f}</b><br>' +
        '  TXST Students: <b>%{customdata[4]:,.0f}</b> (%{customdata[3]:.1f}%)<br><br>' +
        '<b style="color:#E74C3C">Growth Potential (to 15%)</b><br>' +
        '  <b style="font-size:18px">+%{customdata[5]:,.0f} students</b>' +
        '<extra></extra>',
    customdata=tier1_counties[[
        'County', 'College_Ready_Rate', 'Total_All_Schools',
        'TXST_Market_Share', 'TXST'
    ]].assign(Growth_Potential=lambda x: (x['Total_All_Schools'] * 0.15) - x['TXST']).values
))

# TXST
fig2.add_trace(go.Scattergeo(
    lon=[-97.9], lat=[29.9], mode='markers',
    marker=dict(size=40, color='#501214', symbol='star', line=dict(width=4, color='#FFB700')),
    hovertemplate='<b>Texas State University</b><extra></extra>',
    showlegend=False
))

# Elegant county cards
for idx, row in tier1_counties.iterrows():
    centroid = row['geometry'].centroid
    growth = (row['Total_All_Schools'] * 0.15) - row['TXST']
    
    # County name + key metrics
    label_text = (
        f"<b style='font-size:14px'>{row['County']}</b><br>"
        f"<span style='color:#27AE60; font-size:16px'><b>{row['College_Ready_Rate']:.1f}%</b></span> ready<br>"
        f"<span style='color:#3498DB'><b>{int(row['Total_All_Schools']):,}</b></span> students<br>"
        f"<span style='color:#E74C3C'><b>+{int(growth):,}</b></span> potential"
    )
    
    fig2.add_trace(go.Scattergeo(
        lon=[centroid.x], lat=[centroid.y], mode='text',
        text=[label_text],
        textfont=dict(size=10, color='#2C3E50', family='Inter, sans-serif'),
        hoverinfo='skip', showlegend=False
    ))

fig2.update_geos(fitbounds="locations", visible=False, bgcolor='#FAFAFA')

fig2.update_layout(
    title=dict(
        text='<b style="font-size:36px; color:#2C3E50">The Big 6: Prime Recruitment Targets</b><br>' +
             '<span style="font-size:18px; color:#7F8C8D; font-weight:300">High-Performing Markets with Maximum Growth Potential</span><br><br>' +
             '<span style="font-size:28px; color:#E74C3C"><b>+11,017</b></span> <span style="font-size:16px; color:#7F8C8D">student growth opportunity</span>',
        x=0.5, xanchor='center', y=0.97, yanchor='top',
        font=dict(family='Inter, sans-serif')
    ),
    height=1300,
    width=2000,
    paper_bgcolor='#FAFAFA',
    geo=dict(scope='usa', center=dict(lon=-99, lat=31), projection_scale=3.5, bgcolor='#FAFAFA'),
    margin=dict(l=20, r=20, t=180, b=20)
)

fig2.write_html('texas_tier1_SPOTLIGHT.html')
fig2.write_image('texas_tier1_SPOTLIGHT.png', width=3000, height=1950, scale=2)

print("✅ Beautiful Tier 1 spotlight map saved!")

print("\n" + "="*70)
print("✨ GORGEOUS MAPS COMPLETE! ✨")
print("="*70)
print("\nYou now have magazine-quality maps:")
print("  • texas_opportunity_BEAUTIFUL.html/png")
print("  • texas_tier1_SPOTLIGHT.html/png")
print("\nPerfect for your poster! 🎨🏆")

CREATING BEAUTIFUL STRATEGIC OPPORTUNITY MAP

✅ Beautiful main map saved!

Creating Tier 1 spotlight map...
✅ Beautiful Tier 1 spotlight map saved!

✨ GORGEOUS MAPS COMPLETE! ✨

You now have magazine-quality maps:
  • texas_opportunity_BEAUTIFUL.html/png
  • texas_tier1_SPOTLIGHT.html/png

Perfect for your poster! 🎨🏆
